In [ ]:
import sys
sys.path.append('/home/fjargsto/AcousticBEM/Python')

from InteriorHelmholtzSolver3D import *
from InteriorHelmholtzSolver3D_C import *

c         = 344.0 # speed of sound [m/s]
rho       = 1.205 # density of air [kg/m^3]
frequency = 20.0 # frequency [Hz]

print "c   = ", c
print "rho = ", rho
print "f   = ", frequency

In [ ]:
aVertex = np.array([[ 0.000, 0.000, 1.000], 
                    [ 0.000, 0.745, 0.667],
                    [ 0.645, 0.372, 0.667],
                    [ 0.645,-0.372, 0.667],
                    [ 0.000,-0.745, 0.667],
                    [-0.645,-0.372, 0.667],
                    [-0.645, 0.372, 0.667],
                    [ 0.500, 0.866, 0.000],
                    [ 1.000, 0.000, 0.000],
                    [ 0.500,-0.866, 0.000],
                    [-0.500,-0.866, 0.000],
                    [-1.000, 0.000, 0.000],
                    [-0.500, 0.866, 0.000],
                    [ 0.000, 0.745,-0.667],
                    [ 0.645, 0.372,-0.667],
                    [ 0.645,-0.372,-0.667],
                    [ 0.000,-0.745,-0.667],
                    [-0.645,-0.372,-0.667],
                    [-0.645, 0.372,-0.667],
                    [ 0.000, 0.000,-1.000]], dtype=np.float32)

aTriangle = np.array([[ 0,  2,  1], [ 0,  3,  2], [ 0,  4, 3],
                      [ 0,  5,  4], [ 0,  6,  5], [ 0,  1,  6],
                      [ 1,  2,  7], [ 2,  8,  7], [ 2,  3,  8],
                      [ 3,  9,  8], [ 3,  4,  9], [ 4, 10,  9],
                      [ 4,  5, 10], [ 5, 11, 10], [ 5,  6, 11],
                      [ 6, 12, 11], [ 6,  1, 12], [ 1,  7, 12],
                      [ 7, 14, 13], [ 7,  8, 14], [ 8, 15, 14],
                      [ 8,  9, 15], [ 9, 16, 15], [ 9, 10, 16],
                      [10, 17, 16], [10, 11, 17], [11, 18, 17],
                      [11, 12, 18], [12, 13, 18], [12,  7, 13],
                      [13, 14, 19], [14, 15, 19], [15, 16, 19],
                      [16, 17, 19], [17, 18, 19], [18, 13, 19]], dtype=np.int32)

interiorPoints = np.array([[0.500, 0.000, 0.000],
                           [0.000, 0.000, 0.010],
                           [0.000, 0.000, 0.250],
                           [0.000, 0.000, 0.500],
                           [0.000, 0.000, 0.750]], dtype=np.float32)


In [ ]:
centers = (aVertex[aTriangle[:, 0]] + aVertex[aTriangle[:, 1]] + aVertex[aTriangle[:, 2]]) / 3.0

In [ ]:
# Test Problem 1
# Dirichlet boundary condition with phi = sin(k*z)
#

solver = InteriorHelmholtzSolver3D_C(aVertex, aTriangle)

boundaryCondition = BoundaryCondition(aTriangle.shape[0])
boundaryCondition.alpha.fill(1.0)
boundaryCondition.beta.fill(0.0)

k = solver.frequencyToWavenumber(frequency)

boundaryCondition.f[:] = np.sin(k * centers[:,2])

boundaryIncidence = BoundaryIncidence(aTriangle.shape[0])
boundaryIncidence.phi.fill(0.0)
boundaryIncidence.v.fill(0.0)

interiorIncidentPhi = np.zeros(interiorPoints.shape[0], dtype=np.complex64)

boundarySolution = solver.solveBoundary(k, boundaryCondition, boundaryIncidence)

interiorPhi = solver.solveInterior(boundarySolution, interiorIncidentPhi, interiorPoints)

print "Test Problem 1"
print "==============\n"
print boundarySolution
printInteriorSolution(boundarySolution, interiorPhi)


# Test Problem 2
# Neumann boundary condition with v = cos(k*z)
#

boundaryCondition.alpha.fill(0.0)
boundaryCondition.beta.fill(1.0)

for i in range(aTriangle.shape[0]):
    a = aVertex[aTriangle[i, 0]]
    b = aVertex[aTriangle[i, 1]]
    c = aVertex[aTriangle[i, 2]]
    normal = solver.Normal3D(a, b, c)
    boundaryCondition.f[i] = k * np.cos(k * centers[i,2]) * normal[2]

boundarySolution = solver.solveBoundary(k, boundaryCondition, boundaryIncidence)

interiorPhi = solver.solveInterior(boundarySolution, interiorIncidentPhi, interiorPoints)

print "\n\nTest Problem 2"
print "==============\n"
print boundarySolution
printInteriorSolution(boundarySolution, interiorPhi)


# Test Problem 3
# Neumann boundary condition with v = cos(k*z)
#

boundaryCondition.alpha.fill(0.0)
boundaryCondition.beta.fill(1.0)
boundaryCondition.f.fill(0.0)

p = np.array([0.0, 0.0, 0.0], dtype=np.float32)
for i in range(aTriangle.shape[0]):
    q = centers[i, :]
    r = p - q
    R = norm(r)
    boundaryIncidence.phi[i] = np.exp(1.0j * k * R) / (4.0 * np.pi * R)
    a = aVertex[aTriangle[i, 0]]
    b = aVertex[aTriangle[i, 1]]
    c = aVertex[aTriangle[i, 2]]
    normal = solver.Normal3D(a, b, c)
    drbdn = -np.dot(r, normal) / R
    boundaryIncidence.v[i] = drbdn * np.exp(1.0j * k * R)*(1.0j * k * R - 1.0) / \
            (4.0 * np.pi * R * R)

for i in range(interiorPoints.shape[0]):
    q = interiorPoints[i, :]
    r = p - q
    R = norm(r)
    interiorIncidentPhi[i] = np.exp(1.0j * k * R) / (4.0 * np.pi * R)

boundarySolution = solver.solveBoundary(k, boundaryCondition, boundaryIncidence)

interiorPhi = solver.solveInterior(boundarySolution, interiorIncidentPhi, interiorPoints)

print "\n\nTest Problem 3"
print "==============\n"
print boundarySolution
printInteriorSolution(boundarySolution, interiorPhi)


